In [2]:
import mdp 

In [4]:
! python mdp.py < ex.mdp

States: ['S0', 'S1', 'S2']
Actions: ['a', 'b']
Transition from S0 with no action and targets ['S1', 'S2'] with weights [5, 5]
Transition from S1 with action b and targets ['S1', 'S0'] with weights [2, 8]
Transition from S1 with action a and targets ['S2', 'S0', 'S1'] with weights [1, 3, 6]
Transition from S2 with action c and targets ['S0', 'S1'] with weights [5, 5]
